# Session de Recrutement Printemps 2017 - Team Tax Analytics
## Rendu de Guillaume Grelet - guillaume.grelet@telecom-paristech.fr

Date : février 2017


Ce rendu a pour but de fournir une réponse à l'exercice posé par PwC sur la gestion de tableur Excel.  
Il utilise les packages suivante : jupyter, pandas, PyQt, ipywidgets  
  
  
Notes :
Le code suivant est systématiquement commenté en anglais, comme l'usage le veut.  


In [218]:
# Import the necessary packages..

# For the widgets
%gui qt 
from IPython.display import display, Markdown, Latex
from PyQt5.QtWidgets import QFileDialog
from ipywidgets import *

# For the excel file processing...
import pandas as pd
import numpy as np

import jellyfish

In [219]:
def filesearch(dir=None):
    """
    That function allows the user to select a file via a PyQt5 dialog window and return the file name as a str
    """
    if dir is None: dir ="~/" # Default path is ~/
    filename = QFileDialog.getOpenFileName(None)
    return filename[0]

In [220]:
datapath = filesearch("~/")

In [253]:
%%time
print("Processing file...")
data = pd.read_excel(datapath)
print("\nProcessing OK. Stats about processing:\n")

Processing file...

Processing OK. Stats about processing:

CPU times: user 6.75 s, sys: 343 ms, total: 7.09 s
Wall time: 7.82 s


In [254]:
input_label_widget = widgets.Textarea(
    placeholder='Place here the preprocessed labels',
    value='code_hospidiag\nsous_axe',
    disabled=False
)

button1 = widgets.Button(
    description='Save',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)


display(Markdown("## Remplissez ci-dessus la liste des labels standards préprogrammés  "), input_label_widget, button1)

def handle_submit(sender):
    global x
    x = input_label_widget.value
    global input_label_list
    input_label_list = x.split()
    print("input_label_list = ")
    print(input_label_list)
    print("Fichier sauvé")
    return None

def when_button_clicked(button1):
    out = handle_submit(input_label_widget)
    return None 

button1.on_click(when_button_clicked)

## Remplissez ci-dessus la liste des labels standards préprogrammés  

In [225]:
found_labels = list(data)
print("Found labels are the following:")
for str in found_labels:
    print("    " + str)

Found labels are the following:
    axe
    sous_axe
    indic
    code_hospidiag
    libelle


In [226]:
intersection = list(set(found_labels).intersection(input_label_list))
#print(intersection)

existing_label_widget = widgets.Textarea(
    value="\n".join(intersection) + "\n",
    placeholder='Type something',
    disabled=True
)

display(Markdown("## Les labels ci-dessus sont issus du fichier Excel et existent dans la liste prépprogrammée"), existing_label_widget)

## Les labels ci-dessus sont issus du fichier Excel et existent dans la liste prépprogrammée

In [259]:
new_labels = []
for str in found_labels:
    if str not in intersection:
        new_labels.append(str)
                
preprocessed_label_widget = widgets.Textarea(
    value="\n".join(new_labels) + "\n",
    placeholder='Type something',
    disabled=False
)

button2 = widgets.Button(
    description='Ok',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

display(Markdown("## Les labels ci-dessus ne sont pas dans la liste standardisée et sont donc nouveaux. Vous pouvez modifier la liste et les ajouter"), preprocessed_label_widget, button2)

def handle_submit(sender):
    global y
    y = preprocessed_label_widget.value
    global modified_list
    modified_list = y.split()
    print("modified_list = ")
    print(modified_list)
    return None

def when_button_clicked(button):
    out = handle_submit(preprocessed_label_widget)
    return None 

button2.on_click(when_button_clicked)

## Les labels ci-dessus ne sont pas dans la liste standardisée et sont donc nouveaux. Vous pouvez modifier la liste et les ajouter

modified_list = 
['axer', 'indic', 'codes_hospidiag', 'libelle']


In [230]:
mat_size = (len(modified_list) + len(input_label_list)), (len(found_labels))
mat0 = np.array([modified_list + input_label_list])

mat1 = np.array([[None] +  modified_list + input_label_list])
mat2 = np.array([found_labels]).T
mat3 = np.zeros(mat_size)
a, b = 0, 0
for str1 in modified_list + input_label_list:
    for str2 in found_labels:
        mat3[a, b] = round(jellyfish.jaro_distance(str1, str2), 3)
        a = a + 1
    a = 0
    b = b + 1
mat4 = np.concatenate((mat2, mat3), axis = 1)
mat5 = np.concatenate((mat1, mat4), axis = 0)

In [233]:
print(mat5)

[[None 'codes_hospidiag' 'sous_axe' 'label1' 'label2' 'axer']
 ['axe' '0.467' '0.0' '0.667' '0.667' '0.917']
 ['sous_axe' '0.586' '1.0' '0.0' '0.0' '0.0']
 ['indic' '0.344' '0.0' '0.0' '0.0' '0.0']
 ['code_hospidiag' '0.93' '0.512' '0.413' '0.413' '0.44']
 ['libelle' '0.403' '0.423' '0.746' '0.746' '0.464']]


In [294]:
treshold = 0.8
k = 0
a = np.where(mat3>treshold)
matched_labels = found_labels
print("Recherche de labels similaires")
while k < np.maximum(len(a[0]),len(a[1])):
    print("    ------>  " + mat2[a[0][k], 0] + " ressemble à " + mat0[0,a[1][k]])
    for x in matched_labels:
        if x == mat2[a[0][k], 0]:
            matched_labels[matched_labels.index(mat2[a[0][k], 0])] = mat0[0,a[1][k]]
    k = k + 1
print("Labels similaires remplacés")



Recherche de labels similaires
    ------>  axe ressemble à axer
    ------>  sous_axe ressemble à sous_axe
    ------>  code_hospidiag ressemble à codes_hospidiag
Labels similaires remplacés


In [303]:
dico = {}

linker_label_widget = widgets.Text(
    value='Autos',
    placeholder='Type something',
    disabled=False
)

linked_label_widget = widgets.Textarea(
    value="Automobiles\nVoiture\nCar",
    placeholder='Type something',
    disabled=False
)

button3 = widgets.Button(
    description='Save',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

display(Markdown("## Vous pouvez modifier la liste et les ajouter pour lier"), Markdown("## Vous pouvez le faire autant de fois que vous voulez"), linker_label_widget, linked_label_widget, button2)

def handle_submit(sender):
    global linker
    linker = linker_label_widget.value
    global links
    links = linked_label_widget.value.split()
    global dico
    di = {linker: links}
    dico.update(di)
    print(dico)
    return None

def when_button_clicked(button):
    out = handle_submit(linked_label_widget)
    return None 



## Vous pouvez modifier la liste et les ajouter pour lier

## Vous pouvez le faire autant de fois que vous voulez

{'Autos': ['Automobiles', 'Voiture', 'Car']}
